<a href="https://colab.research.google.com/github/efcor/tf-sandbox/blob/main/model-attempt-4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# The tutorial I'm following seems to have some breaking changes with recent
# versions, so force the runtime to downgrade tensorflow via uninstall and
# reinstall. Note: requires a restart of the runtime.
!pip uninstall tensorflow_recommenders
!pip uninstall tensorflow
!pip install tensorflow==2.12
!pip install -q tensorflow_recommenders

Found existing installation: tensorflow 2.17.1
Uninstalling tensorflow-2.17.1:
  Would remove:
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.17.1.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? Y
  Successfully uninstalled tensorflow-2.17.1
INFO: pip is looking at multiple versions of jax to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 585.9/585.9 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 3.2 MB/s eta 0:00:00


In [1]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [14]:
print("TensorFlow version:", tf.__version__)
print("Recommenders version:", tfrs.__version__)
# Good to go if this outputs:
# TensorFlow version: 2.12.0
# Recommenders version: v0.7.3

TensorFlow version: 2.12.0
Recommenders version: v0.7.3


In [4]:
!wget -nc https://raw.githubusercontent.com/tensorflow/examples/master/lite/examples/recommendation/ml/data/example_generation_movielens.py
!python -m example_generation_movielens  --data_dir=data/raw  --output_dir=data/examples  --min_timeline_length=3  --max_context_length=10  --max_context_movie_genre_length=10  --min_rating=2  --train_data_fraction=0.9  --build_vocabs=False

--2024-11-24 17:12:45--  https://raw.githubusercontent.com/tensorflow/examples/master/lite/examples/recommendation/ml/data/example_generation_movielens.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18041 (18K) [text/plain]
Saving to: ‘example_generation_movielens.py’

example_generation_ 100%[===================>]  17.62K  --.-KB/s    in 0.001s  

2024-11-24 17:12:45 (17.5 MB/s) - ‘example_generation_movielens.py’ saved [18041/18041]

2024-11-24 17:12:48.153942: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
I1124 17:12:50.152575 137990725861376 example_generation_movielens.py:460] Downloading and extracting data.
5917549/5917549 [==============================] - 0s 0us/step
I1124 17:12:51.154421 1379907

In [5]:
train_filename = "./data/examples/train_movielens_1m.tfrecord"
train = tf.data.TFRecordDataset(train_filename)

test_filename = "./data/examples/test_movielens_1m.tfrecord"
test = tf.data.TFRecordDataset(test_filename)

feature_description = {
    'context_movie_id': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(0, 10)),
    'context_movie_rating': tf.io.FixedLenFeature([10], tf.float32, default_value=np.repeat(0, 10)),
    'context_movie_year': tf.io.FixedLenFeature([10], tf.int64, default_value=np.repeat(1980, 10)),
    'context_movie_genre': tf.io.FixedLenFeature([10], tf.string, default_value=np.repeat("Drama", 10)),
    'label_movie_id': tf.io.FixedLenFeature([1], tf.int64, default_value=0),
}

def _parse_function(example_proto):
  return tf.io.parse_single_example(example_proto, feature_description)

train_ds = train.map(_parse_function).map(lambda x: {
    "context_movie_id": tf.strings.as_string(x["context_movie_id"]),
    "label_movie_id": tf.strings.as_string(x["label_movie_id"])
})

test_ds = test.map(_parse_function).map(lambda x: {
    "context_movie_id": tf.strings.as_string(x["context_movie_id"]),
    "label_movie_id": tf.strings.as_string(x["label_movie_id"])
})

for x in train_ds.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'context_movie_id': array([b'1409', b'3452', b'3893', b'3566', b'3355', b'2204', b'1212',
       b'908', b'3634', b'2180'], dtype=object),
 'label_movie_id': array([b'2983'], dtype=object)}


In [6]:
movies = tfds.load("movielens/1m-movies", split='train')
movies = movies.map(lambda x: x["movie_id"])
movie_ids = movies.batch(1_000)
unique_movie_ids = np.unique(np.concatenate(list(movie_ids)))

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/3883 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/movielens/1m-movies/incomplete.9CSTRD_0.1.1/movielens-train.tfrecord*...: …

Dataset movielens downloaded and prepared to /root/tensorflow_datasets/movielens/1m-movies/0.1.1. Subsequent calls will reuse this data.


In [7]:
embedding_dimension = 32

query_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
      vocabulary=unique_movie_ids, mask_token=None),
    tf.keras.layers.Embedding(len(unique_movie_ids) + 1, embedding_dimension),
    tf.keras.layers.GRU(embedding_dimension),
])

candidate_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_ids) + 1, embedding_dimension)
])

In [8]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(candidate_model)
)

task = tfrs.tasks.Retrieval(
  metrics=metrics
)

class Model(tfrs.Model):

    def __init__(self, query_model, candidate_model):
        super().__init__()
        self._query_model = query_model
        self._candidate_model = candidate_model

        self._task = task

    def compute_loss(self, features, training=False):
        watch_history = features["context_movie_id"]
        watch_next_label = features["label_movie_id"]

        query_embedding = self._query_model(watch_history)
        candidate_embedding = self._candidate_model(watch_next_label)

        return self._task(query_embedding, candidate_embedding, compute_metrics=not training)

In [9]:
model = Model(query_model, candidate_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [10]:
cached_train = train_ds.shuffle(10_000).batch(12800).cache()
cached_test = test_ds.batch(2560).cache()

In [11]:
model.fit(cached_train, epochs=3)

Epoch 1/3
67/67 [==============================] - 314s 5s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 108111.0470 - regularization_loss: 0.0000e+00 - total_loss: 108111.0470
Epoch 2/3
67/67 [==============================] - 257s 4s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - loss: 101154.7229 - regularization_loss: 0.0000e+00 - total_loss: 101154.7229
Epoch 3/3
67/67 [==============================] - 247s 4s/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - 

In [12]:
model.evaluate(cached_test, return_dict=True)

37/37 [==============================] - 62s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0143 - factorized_top_k/top_5_categorical_accuracy: 0.0767 - factorized_top_k/top_10_categorical_accuracy: 0.1330 - factorized_top_k/top_50_categorical_accuracy: 0.3745 - factorized_top_k/top_100_categorical_accuracy: 0.5111 - loss: 15465.2528 - regularization_loss: 0.0000e+00 - total_loss: 15465.2528


{'factorized_top_k/top_1_categorical_accuracy': 0.014349833130836487,
 'factorized_top_k/top_5_categorical_accuracy': 0.07667458802461624,
 'factorized_top_k/top_10_categorical_accuracy': 0.13302913308143616,
 'factorized_top_k/top_50_categorical_accuracy': 0.37450292706489563,
 'factorized_top_k/top_100_categorical_accuracy': 0.5110502243041992,
 'loss': 9339.1630859375,
 'regularization_loss': 0,
 'total_loss': 9339.1630859375}

In [15]:
# Export the query model. In Colab, this saves
# to /content/movie_recommender_model
tf.saved_model.save(model, 'movie_sequence_model')

# Zip the model folder so I can download it to move it to my server
# !zip -r movie_recommender_model.zip movie_recommender_model

# Load it back; can also be done in TensorFlow Serving.
# loaded = tf.saved_model.load('movie_recommender_model')

# loaded({"user_id": np.array(["42"]), "movie_title": ["Speed (1994)"]}).numpy()

FailedPreconditionError: {{function_node __wrapped__DatasetToGraphV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Failed to serialize the input pipeline graph: ResourceGather is stateful. [Op:DatasetToGraphV2]